## **Purpose**
 In this notebook we will creating a dataset that will join with the twitch dataset.

## **Datasets**
300.csv


In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import os

In [2]:
if not os.path.exists("../data/prep/300.csv"):
    print("Missing dataset file")
else:
    print("Success!")

Success!


In [3]:
df = pd.read_csv("../data/prep/300.csv")

Many games have many different versions in our datarame. For example Grand Theft Auto V is in our dataframe 4 times as its available on 4 different platforms. I want to add the sales of the game for each platform together to get the overall sales for that particular game. I also want to get the average critic and user ratings for each game.

In [4]:
salesdf=df.groupby('name')['global_sales','na_sales','eu_sales','jp_sales','other_sales'].apply(sum).reset_index()

ratingdf=df.groupby('name')['meta_critic_score','meta_user_score'].mean().reset_index() 

yeardf=pd.DataFrame()
yeardf['release_date']=df['release_date']
yeardf['name']=df['name']

I want the rarting scores to be rounded to 1 decimal place.

In [5]:
ratingdf['meta_user_score']=round(ratingdf['meta_user_score'], 1)
ratingdf['meta_critic_score']=round(ratingdf['meta_critic_score'], 1)

Here I join the dataframe containing the sum of the sales with the dataframe containing the average rating scores.

In [6]:
salesdf=salesdf.merge(ratingdf, left_on='name', right_on='name')
salesdf.sample()

,name,global_sales,na_sales,eu_sales,jp_sales,other_sales,meta_critic_score,meta_user_score
3680,Summon Night: Twin Age,130000.0,70000.0,0.0,50000.0,10000.0,73.0,7.2


Now I join that dataframe with the dataframe containing the release date of games

In [7]:
final=salesdf.merge(yeardf, left_on='name', right_on='name')
final.sample()

,name,global_sales,na_sales,eu_sales,jp_sales,other_sales,meta_critic_score,meta_user_score,release_date
312,Astro Bot Rescue Mission,710000.0,220000.0,360000.0,20000.0,110000.0,90.0,8.9,2018-10-02


Finally I remove the duplicate rows in the dataframe

In [8]:
final = final.drop_duplicates(subset='name')

In [9]:
final.head()

,name,global_sales,na_sales,eu_sales,jp_sales,other_sales,meta_critic_score,meta_user_score,release_date
0,.hack//G.U. Last Recode,240000.0,100000.0,40000.0,80000.0,30000.0,76.0,7.8,2017-11-03
1,.hack//Infection Part 1,1270000.0,490000.0,380000.0,260000.0,130000.0,75.0,8.3,2003-02-11
2,007 Legends,340000.0,110000.0,170000.0,0.0,60000.0,41.0,4.1,2012-10-16
3,007 Racing,530000.0,300000.0,200000.0,0.0,30000.0,51.0,4.6,2000-11-20
4,007: Quantum of Solace,3300000.0,1550000.0,1310000.0,40000.0,400000.0,63.5,6.7,2008-11-04


In [10]:
final=final.set_index('name')
final.to_csv("../data/prep/400.csv",sep=",",encoding='utf-8')